In [2]:
import os
import numpy as np
import json
import pandas as pd
import re
pd.set_option('display.max_rows', 500)

df_list = []
# results_dir = '/home/pgajo/projects/def-hsajjad/pgajo/natcode-llm-kgc/results_icl=3/test/base'
# results_dir = '/home/pgajo/projects/def-hsajjad/pgajo/natcode-llm-kgc/results_icl=3/test/fine-tuned/Llama-3.2'
results_dir = '/home/pgajo/projects/def-hsajjad/pgajo/natcode-llm-kgc/results_icl=3_4bit/test/fine-tuned'
for root, dirs, files in os.walk(results_dir):
    for F in files:
        if F.endswith('.json'):
            filename = os.path.join(root, F)
            # print(filename)
            with open(filename, 'r', encoding='utf8') as f:
                data = json.load(f)[:3]
            model_name = filename.split('/')[-1]#[:-5]
            # print(model_name)
            entry = {
                'model': model_name.split('_')[0],
                # 'module': model_name.split('_')[7],
                'dataset': model_name.split('_')[2],
                'lang': model_name.split('_')[3],
                'prompt': model_name.split('_')[4],
                'f1_mean': np.mean([el['F1_Score'] for el in data]).round(3),
                # 'f1_std': np.std([el['F1_Score'] for el in data]).round(3),
                'prec_mean': np.mean([el['Precision'] for el in data]).round(3),
                # 'prec_std': np.std([el['Precision'] for el in data]).round(3),
                'rec_mean': np.mean([el['Recall'] for el in data]).round(3),
                # 'rec_std': np.std([el['Recall'] for el in data]).round(3),
                # 'num_runs': len(data),
            }

            df_list.append(entry)

df = pd.DataFrame(df_list).sort_values(by = ['model', 'dataset', 'lang', 'prompt'])
# print(df['num_runs'].sum())

display(df)
savename = os.path.join(results_dir, 'metrics_complete')
df.to_csv(savename+'.csv', index=False)
df.to_latex(savename+'.tex', index=False, float_format="%.3f")

df_grouped = df.groupby(by=[
                        'model',
                        'dataset',
                        'lang',
                        'prompt',
                        # 'module',
                        ]).agg({'f1_mean': 'mean', 'f1_std': 'mean'}).reset_index()

savename = os.path.join(results_dir, 'metrics_mean')

df_grouped = df_grouped.round(3)

display(df_grouped)

df_grouped.to_csv(savename+'.csv', index=False)
df_grouped.to_latex(savename+'.tex', index=False, float_format="%.3f")

,model,dataset,lang,prompt,f1_mean,prec_mean,rec_mean
2,Meta-Llama-3.1-8B,ade,code,base,0.775,0.778,0.773
15,Meta-Llama-3.1-8B,ade,code,rationale,0.752,0.758,0.745
22,Meta-Llama-3.1-8B,ade,natlang,base,0.772,0.785,0.760
14,Meta-Llama-3.1-8B,ade,natlang,rationale,0.751,0.755,0.747
19,Meta-Llama-3.1-8B,conll04,code,base,0.584,0.580,0.590
11,Meta-Llama-3.1-8B,conll04,code,rationale,0.548,0.558,0.539
9,Meta-Llama-3.1-8B,conll04,natlang,base,0.566,0.544,0.590
4,Meta-Llama-3.1-8B,conll04,natlang,rationale,0.578,0.581,0.574
0,Meta-Llama-3.1-8B,scierc,code,base,0.341,0.345,0.337
23,Meta-Llama-3.1-8B,scierc,code,rationale,0.289,0.287,0.291


KeyError: "Column(s) ['f1_std'] do not exist"